# 2-6 損失関数の実装

## jaccard 係数を用いた match 関数の動作
はじめに，すべてのデフォルトボックスから正解データと物体クラスが一致しており座標情報の近いものを抽出する関数 match を定義する．
正解のバウンディングボックスと近いデフォルトボックスの抽出には jaccard 係数を用いる．
jaccard 係数は次の図のように計算できる．



バウンディングボックスとデフォルトボックスの間の jaccard 係数は，2つのボックスの総面積に対するかぶっている部分の面積で計算できる．
2つのボックスが完全にかぶっていれば jaccard 係数は1に，完全に外れている場合には0となる．  
この jaccard 係数を用いて訓練データの正解バウンディングボックスとの係数の値が閾値（ここでは jaccard_thresh = 0.5）以上のデフォルトボックスを Positive Default Box とする．



実装においては次のように処理を行う．
まず，Positive なデフォルトボックスがない場合，何もない部分を背景として認識させるため，背景ラベルである0を与える．  
jaccard 係数が閾値以上のデフォルトボックスだある場合，それを Positive として jaccard 係数が最大になる正解バウンディングボックスの物体クラスを教師データの正解クラスとし，Positive なデフォルトボックスから正解のバウンディングボックスに変形させるオフセット値を loc の教師データとする．
SSD ではデフォルトボックスの座標情報と検出した物体クラスを別々に扱っている点に注意する必要がある．  
match 関数の実装はかなり複雑なため，実装済みの"./utils/match.py"を流用する．
オフセット情報とラベル情報の正解教師データとして，それぞれ loc_t と conf_t_label を返すようになっている．
背景クラスをインデックス0とするため，VOC2012 で用意されている物体クラスのインデックスを+1している．

## Hard Negative Mining
